In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import threading
import csv
import time
import folium
import cv2
import math
import socket

# Plot values in opencv program
from Realtime_Plotter import Realtime_Plotter


In [2]:
def is_socket_closed(sock: socket.socket) -> bool:
    # https://stackoverflow.com/questions/48024720/python-how-to-check-if-socket-is-still-connected
    try:
        # this will try to read bytes without blocking and also without removing them from buffer (peek only)
        data = sock.recv(16, socket.MSG_DONTWAIT | socket.MSG_PEEK)
        if len(data) == 0:
            return True
    except BlockingIOError:
        return False  # socket is open and reading from it would block
    except ConnectionResetError:
        return True  # socket was closed for some other reason
    except Exception as e:
        print("unexpected exception when checking if a socket is closed")
        return False
    return False

In [3]:
class PWM_GUI:
    throttles = []
    steers = []
    statuses = []
    
    max_vals = -1
    
    def __init__(self, max_vals):
        self.max_vals = max_vals
        self.steering_plotter = Realtime_Plotter(plot_width=200, plot_height=400, min_val=1000, max_val=2000, label="Steering")
        self.throttle_plotter = Realtime_Plotter(plot_width=200, plot_height=400, min_val=1000, max_val=2000, label="Throttle")
        self.df = pd.DataFrame(columns=['header', 'status', 'steer', 'throttle'])

    def ingest_line(self, line):
            self.df.loc[len(self.df)] = line
            self.statuses.append(line[1])
            self.steers.append(line[2])
            self.throttles.append(line[3])
            
            while len(self.steers) > self.max_vals:
                self.steers.pop(0)
            while len(self.throttles) > self.max_vals:
                self.throttles.pop(0)
            while len(self.statuses) > self.max_vals:
                self.statuses.pop(0)
            
            self.steering_plotter.build_plot(line[2])
            self.throttle_plotter.build_plot(line[3])
            
            disp = np.hstack((np.zeros((400,50,3)), self.steering_plotter.plot, np.zeros((400,50,3)), self.throttle_plotter.plot, np.zeros((400,50,3))))
            cv2.imshow("Steering - Throttle", disp)
            key = cv2.waitKey(10)
            
            if key == 113:
                return False
            else:
                return True        

In [4]:
class GPS_GUI:
    lats = []
    lons = []
    alts = []
    speeds = []
    headings = []
    
    max_vals = -1
    
    def __init__(self, max_vals):
        self.max_vals = max_vals
        self.speed_plotter = Realtime_Plotter(plot_width=200, plot_height=400, min_val=-1.0, max_val=30.0, label="Speed")
        self.heading_plotter = Realtime_Plotter(plot_width=200, plot_height=400, min_val=-10.0, max_val=370.0, label="Heading")
        self.df = pd.DataFrame(columns=['header', 'lat', 'lon', 'alt', 'speed', 'heading', 'fusion_calibration'])

    def ingest_line(self, line):
            line[1] = float(line[1]) / 10**7
            line[2] = float(line[2]) / 10**7
            line[3] = float(line[3]) / 10**3
            line[4] = float(line[4]) / 10**3
            line[5] = float(line[5]) / 10**5
            self.df.loc[len(self.df)] = line
            
            self.lats.append(line[1])
            self.lons.append(line[2])
            self.alts.append(line[3])
            self.speeds.append(line[4])
            self.headings.append(line[5])
            
            while len(self.lats) > self.max_vals:
                self.lats.pop(0)
            while len(self.lons) > self.max_vals:
                self.lons.pop(0)
            while len(self.alts) > self.max_vals:
                self.alts.pop(0)
            while len(self.speeds) > self.max_vals:
                self.speeds.pop(0)
            while len(self.headings) > self.max_vals:
                self.headings.pop(0)
            
            self.speed_plotter.build_plot(line[4])
            self.heading_plotter.build_plot(line[5])
            
            disp = np.hstack((np.zeros((400,50,3)), self.speed_plotter.plot, np.zeros((400,50,3)), self.heading_plotter.plot, np.zeros((400,50,3))))
            cv2.imshow("Speed - Heading", disp)
            key = cv2.waitKey(10)
            
            if key == 113:
                return False
            else:
                return True 
        

In [5]:

gps_ui = GPS_GUI(max_vals=20)
pwm_ui = PWM_GUI(max_vals=20)

ola_log = pd.DataFrame(columns=['date', 'time', 'q1', 'q2', 'q3', 'x0', 'x1', 'x2', 'x3', 'x4', 'x5', 'x6', 'x7', 'x8', 'x9', 'x10', 'x11'])


# opening the CSV file
with open('sample_gui_socket_data.csv', mode ='r')as file:
   
  # reading the CSV file
  csvFile = csv.reader(file)
 
  # displaying the contents of the CSV file
  for line in csvFile:
        # print(line)
        if len(line) < 2:
            continue
        elif '$GPS' in line[0]:
            keep_running = gps_ui.ingest_line(line)
        elif '/2000' in line[0]:
            # keep_running = ola_ui.ingest_line(line)
            ola_log.loc[len(ola_log)] = line
        elif 'PWM' in line[0]:
            keep_running = pwm_ui.ingest_line(line)
        else:
            print("line with error: ", line[0])
            
        if not keep_running:
            break


# display(pwm_log)

cv2.destroyAllWindows()

In [6]:
# Import socket module
import socket     
import time  
import sys
          

gps_ui = GPS_GUI(max_vals=20)
pwm_ui = PWM_GUI(max_vals=20)

ola_log = pd.DataFrame(columns=['date', 'time', 'q1', 'q2', 'q3', 'x0', 'x1', 'x2', 'x3', 'x4', 'x5', 'x6', 'x7', 'x8', 'x9', 'x10', 'x11'])


s_time = time.time()
c_time = time.time() - s_time
max_duration = 10.0


# Define the port on which you want to connect
port = 39468 

while c_time < max_duration:
    c_time = time.time() - s_time
    # Create a socket object
    sock = socket.socket()
    try:
        # sock.connect(('192.168.1.46', port))
        sock.connect(('192.168.1.137', port))
        
        # receive data from the server and decoding to get the string.
        for i in range(100):
            line = sock.recv(1024).decode().split(",")
            print (line)
            if len(line) < 2:
                continue
            elif '$GPS' in line[0]:
                keep_running = gps_ui.ingest_line(line)
            elif '/2000' in line[0]:
                pass
                # keep_running = ola_ui.ingest_line(line)
                # ola_log.loc[len(ola_log)] = line
            elif 'PWM' in line[0]:
                print("pwm: ", line)
                keep_running = pwm_ui.ingest_line(line)
            else:
                print("line with error: ", line[0])
                
            if not keep_running:
                break
            # print(sys.getsizeof(data))
            # print("i: ", i)
            # s.send("1".encode())
        sock.close()
    except ConnectionRefusedError:
        print("Connection not available yet - refused...")
        time.sleep(1)
        sock.close()
    
# close the connection
sock.close() 

display(gps_ui.df)
cv2.destroyAllWindows()

TimeoutError: [WinError 10060] A connection attempt failed because the connected party did not properly respond after a period of time, or established connection failed because connected host has failed to respond

In [ ]:
# https://python-visualization.github.io/folium/quickstart.html
m = folium.Map(location=[gps_ui.df.lat[0], gps_ui.df.lon[0]], zoom_start=20)
trail_coordinates = []
for i in range(len(gps_ui.df)):
    trail_coordinates.append((gps_ui.df.lat[i], gps_ui.df.lon[i]))


folium.PolyLine(trail_coordinates, tooltip="Coast").add_to(m)
folium.Circle(
    radius=5,
    location=[gps_ui.df.lat[0], gps_ui.df.lon[0]],
    popup="Start",
    color="crimson",
    fill=False).add_to(m)

folium.Circle(
    radius=1,
    location=[gps_ui.df.lat[0], gps_ui.df.lon[0]],
    popup="Start",
    color="crimson",
    fill=True).add_to(m)

display(m)